In [1]:
from keras.models import Model, load_model  

/home/eric/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# from mobilenet_v2 import relu6
# from mobilenet_v2 import DepthwiseConv2D
# model = load_model("./trained_model/mobilenet_v2/model_weights.25-0.7927.hdf5",custom_objects={'relu6': relu6,'DepthwiseConv2D':DepthwiseConv2D})
# model=load_model('./trained_model/inception_resnet_v2/model_weights.13-0.8069.hdf5')
model=load_model('./trained_model/resnet50/resnet50.36-0.8762.hdf5')
model.compile(loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])
print(model.summary())

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 360, 360, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 366, 366, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 180, 180, 64) 9472        zero_padding2d_1[0][0]           
________________________________________________________________________________________

In [3]:
from keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(1.0/255)
test_dir='/home/eric/data/plant/ai_challenger_pdr2018_testa_20181023/AgriculturalDisease_testA'

classes=[]
with open("labels.txt","r") as f:
    for line in f.readlines():
        classes.append(line.strip("\n").split(" ")[0])

generator=test_datagen.flow_from_directory(
                         test_dir,
                         target_size=(360,360),
                         batch_size=1,
                         class_mode=None,
                         shuffle=False,
                         seed=42
                    )


Found 4514 images belonging to 1 classes.


In [4]:
probability=model.predict_generator(generator,steps=10,verbose=1)

/home/eric/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:536: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


4514/4514 [==============================] - 102s 23ms/step


In [8]:
print(type(probability.shape))

<class 'tuple'>


In [15]:
print(type(probability[:,0]))

<class 'numpy.ndarray'>


In [14]:
print(type(generator.filenames))

<class 'list'>


In [19]:
data={}
data['filename']=generator.filenames
for i in range(probability.shape[1]):
    data[i]=probability[:,i]


In [20]:
import pandas as pd
result=pd.DataFrame(data)
result.to_csv('submit.csv',index=False)

In [21]:
probability.shape

(4514, 61)

In [9]:
import numpy as np
sorted_arr=np.argsort(probability)
sorted_arr[0][-1]

0

In [10]:
generator.reset()
pred=model.predict_generator(generator,steps=2,verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)

  10/4540 [..............................] - ETA: 2:20

/home/eric/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:536: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


4540/4540 [==============================] - 99s 22ms/step


In [11]:
predicted_class_indices[1000:1100]

array([23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24,
       24, 24, 24, 25, 25, 25, 25, 26, 26, 25, 25, 25, 25, 25, 25, 25, 26,
       25, 25, 25, 25, 26, 25, 25, 25, 26, 26, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 26, 25, 25, 25, 26, 26, 25, 25, 26, 25, 26])

In [12]:
import pandas as pd
filenames=generator.filenames
result=pd.DataFrame({'FileName':filenames,
                    'Predictions':predicted_class_indices})
result.to_csv('results.csv',index=False)

In [60]:
from tqdm import tqdm
result = []
for i in tqdm(range(len(filenames))):
    temp_dict = {}
    temp_dict['image_id'] = filenames[i].split('/')[-1]
    temp_dict['disease_class'] = int(predicted_class_indices[i])
    result.append(temp_dict)
#     break
#     print('image %s is %d' % (test_image, sorted_arr[:,-1][0]))
import json
with open('submit.json', 'w') as f:
    json.dump(result, f)
    print('write result json, num is %d' % len(result)) 

100%|██████████| 4959/4959 [00:00<00:00, 703995.72it/s]

write result json, num is 4959


In [4]:
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
import cv2, numpy as np
from keras.applications import imagenet_utils
preprocess = imagenet_utils.preprocess_input
# image_path = '/home/eric/data/ai_challenger_plant_train_20170904/AgriculturalDisease_validationset/data/1/43234193db4aefa1245592ab36d6c946.jpg'
image_path='/home/eric/data/ai_challenger_plant_train_20170904/AgriculturalDisease_trainingset/data/1/fac4a730a7721beb7403371b1e627277.jpg'
img_width, img_height = 224, 224
image = cv2.resize(cv2.imread(image_path), (img_width, img_height)).astype(np.float32)
# image = load_img(image_path, target_size=(img_width,img_height))
image /= 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
# image=image/255.0
# image = preprocess(image)
predictions=model.predict(image)
image

array([[[[0.6117647 , 0.50980395, 0.5647059 ],
         [0.59607846, 0.49411765, 0.54901963],
         [0.58431375, 0.48235294, 0.53333336],
         ...,
         [0.6509804 , 0.5568628 , 0.59607846],
         [0.63529414, 0.5411765 , 0.5803922 ],
         [0.6156863 , 0.52156866, 0.56078434]],

        [[0.6117647 , 0.50980395, 0.5647059 ],
         [0.6117647 , 0.50980395, 0.5647059 ],
         [0.6156863 , 0.5137255 , 0.5686275 ],
         ...,
         [0.654902  , 0.56078434, 0.6       ],
         [0.6901961 , 0.59607846, 0.63529414],
         [0.7294118 , 0.63529414, 0.6745098 ]],

        [[0.6       , 0.5019608 , 0.5529412 ],
         [0.6       , 0.49803922, 0.5529412 ],
         [0.6       , 0.49803922, 0.5529412 ],
         ...,
         [0.61960787, 0.5254902 , 0.5647059 ],
         [0.6       , 0.5058824 , 0.54509807],
         [0.5921569 , 0.49803922, 0.5372549 ]],

        ...,

        [[0.627451  , 0.5372549 , 0.5921569 ],
         [0.63529414, 0.54509807, 0.6       ]

In [5]:
predictions

array([[2.63650800e-05, 6.03371357e-13, 1.20812709e-20, 1.86288873e-10,
        1.99314843e-21, 3.47929380e-28, 1.48060164e-04, 8.90452788e-15,
        5.04146641e-15, 2.97245549e-07, 1.13840323e-28, 2.17847560e-27,
        4.10611257e-02, 9.35056359e-02, 8.74694761e-12, 4.18123836e-03,
        3.79479825e-01, 1.20389727e-23, 3.44746428e-22, 1.18160296e-23,
        8.29511890e-08, 1.31631923e-08, 1.15668712e-29, 2.71920291e-28,
        3.41682266e-10, 3.31076592e-01, 1.12855975e-02, 1.78705249e-02,
        1.37580892e-05, 4.99541522e-04, 6.18043464e-11, 2.02263835e-12,
        8.01639571e-13, 1.30806398e-02, 1.53369454e-22, 1.19313218e-20,
        1.92652838e-12, 6.20452311e-13, 3.47465385e-19, 6.21299118e-24,
        7.47540972e-13, 5.50675607e-17, 1.20259459e-14, 8.09082218e-16,
        8.61733072e-17, 4.81308844e-26, 3.48433088e-11, 4.14584008e-16,
        1.95736112e-03, 2.47716140e-02, 1.56667799e-08, 3.11523946e-13,
        5.66839676e-17, 6.45449267e-27, 8.39499650e-12, 2.152805

In [6]:
sorted_arr=np.argsort(predictions)
sorted_arr[:,-1][0]

16

In [7]:
import os
test_dir='/home/eric/data/ai_challenger_plant_train_20170904/AgriculturalDisease_testA/images'
# test_dir='/home/eric/data/ai_challenger_plant_train_20170904/AgriculturalDisease_validationset/images'
test_images = os.listdir(test_dir)
test_images[0]

'5613dcbdddd99c41446bb728592b4068.jpg'

In [8]:
from tqdm import tqdm
result = []
for test_image in tqdm(test_images):
    temp_dict = {}
    img = cv2.resize(cv2.imread(os.path.join(test_dir,test_image)), (img_width, img_height)).astype(np.float32)
    img /= 255.0
    image = img_to_array(img)
    image = np.expand_dims(image, axis=0)
    predictions=model.predict(image)
    sorted_arr=np.argsort(predictions)
    temp_dict['image_id'] = test_image
    temp_dict['disease_class'] = int(sorted_arr[:,-1][0])
    result.append(temp_dict)
#     break
#     print('image %s is %d' % (test_image, sorted_arr[:,-1][0]))
    

100%|██████████| 4959/4959 [01:34<00:00, 52.38it/s]


In [9]:
import json
with open('submit.json', 'w') as f:
    json.dump(result, f)
    print('write result json, num is %d' % len(result))

write result json, num is 4959
